### Import Packages

In [55]:
# Basics
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000)
from datetime import datetime as dt

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

### Load the data

In [56]:
!pwd

/Users/Alice/Documents/Omdena/Local Chapters - Chennai/chennai-india-power-outage/src/tasks/task-06-Model Building


In [57]:
# load in outage data as df
df = pd.read_csv('../../data/cleaned data/demand_supply_monthly.csv')
df.head()

,Requirement(MU/DAY),Energy_met(MU/DAY),Surplus(+)/Deficit(-)(MU/DAY),Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU),Requirement(MW),Peak_Demand_Met5(MW),Surplus(+)/Deficit(-)(MW),%Shortage(MW),name_report,year,month,date_published,month_published,year_published
0,354.0,354.0,-0.9,10632.0,10605.0,-26.0,-0.2,17646.0,17563.0,-83.0,-0.5,Monthly_Report_Apr_2022,2022,4,2022-05-23,5,2022
1,337.0,337.0,0.0,10459.0,10458.0,-1.0,0.0,16906.0,16906.0,0.0,0.0,Monthly_Report_May_2022,2022,5,2022-06-23,6,2022
2,344.0,343.0,-0.3,10657.0,10648.0,-9.0,-0.1,17196.0,17196.0,0.0,0.0,Monthly_Report_Mar_2022,2022,3,2022-04-25,4,2022
3,286.0,286.0,0.0,8873.0,8873.0,1.0,0.0,15290.0,15290.0,0.0,0.0,Monthly_Report_Jan_2022,2022,1,2022-02-23,2,2022
4,245.0,245.0,0.0,7362.0,7360.0,-1.0,0.0,13501.0,13480.0,-21.0,-0.2,Monthly_Report_Nov_2021,2021,11,2021-12-23,12,2021


In [58]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Requirement(MU/DAY)            122 non-null    float64
 1   Energy_met(MU/DAY)             122 non-null    float64
 2   Surplus(+)/Deficit(-)(MU/DAY)  122 non-null    float64
 3   Requirement(MU)                122 non-null    float64
 4   Energy_met(MU)                 122 non-null    float64
 5   Surplus(+)/Deficit(-)(MU)      122 non-null    float64
 6   %Shortage(MU)                  122 non-null    float64
 7   Requirement(MW)                122 non-null    float64
 8   Peak_Demand_Met5(MW)           122 non-null    float64
 9   Surplus(+)/Deficit(-)(MW)      122 non-null    float64
 10  %Shortage(MW)                  122 non-null    float64
 11  name_report                    122 non-null    object 
 12  year                           122 non-null    int

In [59]:
df.describe()

,Requirement(MU/DAY),Energy_met(MU/DAY),Surplus(+)/Deficit(-)(MU/DAY),Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU),Requirement(MW),Peak_Demand_Met5(MW),Surplus(+)/Deficit(-)(MW),%Shortage(MW),year,month,month_published,year_published
count,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000
mean,283.758197,276.782787,-6.989344,8631.884426,8419.755738,-212.110656,-2.718033,14066.011475,13849.101639,-216.893443,-1.734426,2016.836066,6.467213,6.598361,2017.131148
std,27.948258,36.694300,14.425527,864.073361,1127.604563,438.266568,5.696266,1470.967073,1771.175005,583.523308,4.757991,2.969155,3.447803,3.271868,2.721056
min,217.000000,177.000000,-60.000000,6511.100000,5302.000000,-1860.000000,-24.000000,10161.000000,8518.000000,-3745.000000,-28.000000,2012.000000,1.000000,1.000000,2013.000000
25%,264.500000,254.000000,-7.000000,7994.750000,7759.750000,-216.525000,-2.825000,12969.000000,12732.000000,-161.250000,-1.250000,2014.000000,4.000000,4.000000,2014.000000
50%,280.500000,279.050000,-0.200000,8534.100000,8431.000000,-5.450000,-0.100000,14290.500000,14225.500000,-23.000000,-0.200000,2017.000000,6.000000,7.000000,2017.000000
75%,302.875000,302.800000,0.000000,9201.150000,9201.650000,-0.750000,0.000000,15024.500000,15007.250000,0.000000,0.000000,2019.000000,9.000000,9.000000,2019.000000
max,354.000000,354.000000,0.100000,10657.000000,10648.000000,3.000000,0.000000,17646.000000,17563.000000,2360.000000,20.700000,2022.000000,12.000000,12.000000,2022.000000


## Time Series Data

### Prepare time series data

In [60]:
df.head()

,Requirement(MU/DAY),Energy_met(MU/DAY),Surplus(+)/Deficit(-)(MU/DAY),Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU),Requirement(MW),Peak_Demand_Met5(MW),Surplus(+)/Deficit(-)(MW),%Shortage(MW),name_report,year,month,date_published,month_published,year_published
0,354.0,354.0,-0.9,10632.0,10605.0,-26.0,-0.2,17646.0,17563.0,-83.0,-0.5,Monthly_Report_Apr_2022,2022,4,2022-05-23,5,2022
1,337.0,337.0,0.0,10459.0,10458.0,-1.0,0.0,16906.0,16906.0,0.0,0.0,Monthly_Report_May_2022,2022,5,2022-06-23,6,2022
2,344.0,343.0,-0.3,10657.0,10648.0,-9.0,-0.1,17196.0,17196.0,0.0,0.0,Monthly_Report_Mar_2022,2022,3,2022-04-25,4,2022
3,286.0,286.0,0.0,8873.0,8873.0,1.0,0.0,15290.0,15290.0,0.0,0.0,Monthly_Report_Jan_2022,2022,1,2022-02-23,2,2022
4,245.0,245.0,0.0,7362.0,7360.0,-1.0,0.0,13501.0,13480.0,-21.0,-0.2,Monthly_Report_Nov_2021,2021,11,2021-12-23,12,2021


In [61]:
# data frame is clean, no nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Requirement(MU/DAY)            122 non-null    float64
 1   Energy_met(MU/DAY)             122 non-null    float64
 2   Surplus(+)/Deficit(-)(MU/DAY)  122 non-null    float64
 3   Requirement(MU)                122 non-null    float64
 4   Energy_met(MU)                 122 non-null    float64
 5   Surplus(+)/Deficit(-)(MU)      122 non-null    float64
 6   %Shortage(MU)                  122 non-null    float64
 7   Requirement(MW)                122 non-null    float64
 8   Peak_Demand_Met5(MW)           122 non-null    float64
 9   Surplus(+)/Deficit(-)(MW)      122 non-null    float64
 10  %Shortage(MW)                  122 non-null    float64
 11  name_report                    122 non-null    object 
 12  year                           122 non-null    int

In [62]:
df.index

RangeIndex(start=0, stop=122, step=1)

In [63]:
#create unique dates using year and month column
df['year']=df['year'].astype(str)
df['month']=df['month'].astype(str)
df['ts_date'] = df[['year', 'month']].agg('-'.join, axis=1)
df['ts_date'] = pd.to_datetime(df['ts_date'], format = '%Y-%m')

In [64]:
df.head()

,Requirement(MU/DAY),Energy_met(MU/DAY),Surplus(+)/Deficit(-)(MU/DAY),Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU),Requirement(MW),Peak_Demand_Met5(MW),Surplus(+)/Deficit(-)(MW),%Shortage(MW),name_report,year,month,date_published,month_published,year_published,ts_date
0,354.0,354.0,-0.9,10632.0,10605.0,-26.0,-0.2,17646.0,17563.0,-83.0,-0.5,Monthly_Report_Apr_2022,2022,4,2022-05-23,5,2022,2022-04-01
1,337.0,337.0,0.0,10459.0,10458.0,-1.0,0.0,16906.0,16906.0,0.0,0.0,Monthly_Report_May_2022,2022,5,2022-06-23,6,2022,2022-05-01
2,344.0,343.0,-0.3,10657.0,10648.0,-9.0,-0.1,17196.0,17196.0,0.0,0.0,Monthly_Report_Mar_2022,2022,3,2022-04-25,4,2022,2022-03-01
3,286.0,286.0,0.0,8873.0,8873.0,1.0,0.0,15290.0,15290.0,0.0,0.0,Monthly_Report_Jan_2022,2022,1,2022-02-23,2,2022,2022-01-01
4,245.0,245.0,0.0,7362.0,7360.0,-1.0,0.0,13501.0,13480.0,-21.0,-0.2,Monthly_Report_Nov_2021,2021,11,2021-12-23,12,2021,2021-11-01


In [65]:
# set index
df = df.set_index(['ts_date'])

In [66]:
df.head()

,Requirement(MU/DAY),Energy_met(MU/DAY),Surplus(+)/Deficit(-)(MU/DAY),Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU),Requirement(MW),Peak_Demand_Met5(MW),Surplus(+)/Deficit(-)(MW),%Shortage(MW),name_report,year,month,date_published,month_published,year_published
ts_date,,,,,,,,,,,,,,,,,
2022-04-01,354.0,354.0,-0.9,10632.0,10605.0,-26.0,-0.2,17646.0,17563.0,-83.0,-0.5,Monthly_Report_Apr_2022,2022,4,2022-05-23,5,2022
2022-05-01,337.0,337.0,0.0,10459.0,10458.0,-1.0,0.0,16906.0,16906.0,0.0,0.0,Monthly_Report_May_2022,2022,5,2022-06-23,6,2022
2022-03-01,344.0,343.0,-0.3,10657.0,10648.0,-9.0,-0.1,17196.0,17196.0,0.0,0.0,Monthly_Report_Mar_2022,2022,3,2022-04-25,4,2022
2022-01-01,286.0,286.0,0.0,8873.0,8873.0,1.0,0.0,15290.0,15290.0,0.0,0.0,Monthly_Report_Jan_2022,2022,1,2022-02-23,2,2022
2021-11-01,245.0,245.0,0.0,7362.0,7360.0,-1.0,0.0,13501.0,13480.0,-21.0,-0.2,Monthly_Report_Nov_2021,2021,11,2021-12-23,12,2021


In [67]:
df.columns

Index(['Requirement(MU/DAY)', 'Energy_met(MU/DAY)',
       'Surplus(+)/Deficit(-)(MU/DAY)', 'Requirement(MU)', 'Energy_met(MU)',
       'Surplus(+)/Deficit(-)(MU)', '%Shortage(MU)', 'Requirement(MW)',
       'Peak_Demand_Met5(MW)', 'Surplus(+)/Deficit(-)(MW)', '%Shortage(MW)',
       'name_report', 'year', 'month', 'date_published', 'month_published',
       'year_published'],
      dtype='object')

In [68]:
# delete all extra columns from the dataframe
ts = df.drop(columns=['Requirement(MU/DAY)', 'Energy_met(MU/DAY)',
       'Surplus(+)/Deficit(-)(MU/DAY)', 'Requirement(MW)',
       'Peak_Demand_Met5(MW)', 'Surplus(+)/Deficit(-)(MW)', '%Shortage(MW)',
       'name_report', 'year', 'month', 'date_published', 'month_published',
       'year_published'])

In [69]:
ts.head()

,Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU)
ts_date,,,,
2022-04-01,10632.0,10605.0,-26.0,-0.2
2022-05-01,10459.0,10458.0,-1.0,0.0
2022-03-01,10657.0,10648.0,-9.0,-0.1
2022-01-01,8873.0,8873.0,1.0,0.0
2021-11-01,7362.0,7360.0,-1.0,0.0


## Investigating Time Series with Datetime Objects

Datetime objects include aspects of the date as attributes, like month and year:

In [70]:
# Entire
ts.index[0]

Timestamp('2022-04-01 00:00:00')

In [71]:
# Day
ts.index[0].day

1

In [72]:
# Month
ts.index[0].month

4

In [73]:
# Year
ts.index[0].year

2022

In [74]:
# Sort and reset index
ts = ts.sort_index()

In [75]:
ts.head()

,Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU)
ts_date,,,,
2012-04-01,7707.0,5940.0,-1767.0,-23.0
2012-05-01,7605.0,6649.0,-956.0,-13.0
2012-06-01,7928.0,6893.0,-1035.0,-13.0
2012-07-01,7981.0,7272.0,-709.0,-9.0
2012-08-01,7851.0,6773.0,-1078.0,-14.0
